<a href="https://colab.research.google.com/github/seanmcalevey/side_project/blob/master/trump_tweet_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import re

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AHKSuuQcp9JzOrLfYXxJNOubX8SjnC3ccNmm5z-UKtIV84RzSPjSjw
Mounted at /content/drive/


In [2]:
drive_path = '/content/drive/My Drive/'
folder_path = 'kaggle_datasets/trump_tweets/'
filename = 'trumptweets.csv'

master_df = pd.read_csv(drive_path + folder_path + filename)
df = master_df.copy()
df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags,geo
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 20:54:25,500,868,NaN,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-05 03:00:10,33,273,NaN,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 15:38:08,12,18,NaN,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 22:40:15,11,24,NaN,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 16:07:28,1399,1965,NaN,NaN,NaN


In [3]:
pd.set_option('max_colwidth', 150)
df['content'].sample(5)

15377                                                        The VA scandal shows the fatal ineptitude of big central planning government. When will we learn?
38965    The Wall is going up very fast despite total Obstruction by Democrats in Congress, and elsewhere! https://www.instagram.com/p/B1t2uUeBXdU/?igshid=...
35445    I have no doubt that, if the attack on Dr. Ford was as bad as she says, charges would have been immediately filed with local Law Enforcement Autho...
18788                                                   " @ iluvmorso: @ realDonaldTrump can't wait to see the Apprentice - it's back finally" On January 4th.
38425                                                                               Going with First Lady to pay our respects to Justice Stevens. Leaving now!
Name: content, dtype: object

In [70]:
def clean_text(text_list):

  import re
  import sys
  sys.path.append('/content/drive/My Drive/py_functions')
  from summarizer_func_lib import replace_contraction
  sys.path.remove('/content/drive/My Drive/py_functions')
  from bs4 import BeautifulSoup

  cleaned_reviews = []
  for review in text_list:
    cleaned_review = BeautifulSoup(review, 'lxml').get_text() # clean all html from text
    cleaned_review = re.sub('[^A-Za-z#@/.!\s\'\-]', '', cleaned_review) # basic cleaning
    cleaned_review = re.sub('http\w*', ' ', cleaned_review) # clean http(s) html
    cleaned_review = re.sub('www\w*', ' ', cleaned_review) # clean website addresses
    cleaned_review = re.sub('.+\.com.+', ' ', cleaned_review) # " "
    cleaned_review = re.sub('@', ' @ ', cleaned_review)
    cleaned_review = re.sub('#', ' # ', cleaned_review)
    cleaned_review = re.sub('/', ' ', cleaned_review)
    cleaned_review = re.sub('-', ' ', cleaned_review)

    # Split off periods, commas, and exclamation points as their own tokens, and search for contractions and replace them
    review_words = []
    for word in cleaned_review.split():
      word = word.lower()
      if re.search('\.+', word):
        period_loc = re.search('\.+', word).span()[0]
        tmp_string = word[:period_loc]
        proc_words = replace_contraction(tmp_string)
        for w in proc_words.split():
          review_words.append(w)
        review_words.append('.')
      elif re.search(',', word):
        comma_loc = re.search(',', word).span()[0]
        tmp_string = word[:comma_loc]
        proc_words = replace_contraction(tmp_string)
        for w in proc_words.split():
          review_words.append(w)
        review_words.append(',')
      elif re.search('!', word):
        exclamation_loc = re.search('!', word).span()[0]
        tmp_string = word[:exclamation_loc]
        proc_words = replace_contraction(tmp_string)
        for w in proc_words.split():
          review_words.append(w)
        review_words.append('!')
      else:
        proc_words = replace_contraction(word)
        for w in proc_words.split():
          review_words.append(w)
    combined_review = ' '.join(review_words)
    cleaned_reviews.append(combined_review)

  return cleaned_reviews

In [71]:
tweets = list(df['content'])
cleaned_tweets = clean_text(tweets)
words = [word for tweet in cleaned_tweets for word in tweet.split()]
words[:10]

['be', 'sure', 'to', 'tune', 'in', 'and', 'watch', 'donald', 'trump', 'on']

In [72]:
from tensorflow.keras.preprocessing.text import Tokenizer

num_words = 5000

tokenizer = Tokenizer(num_words=num_words, filters='')
tokenizer.fit_on_texts(words)
encoded = tokenizer.texts_to_sequences(words)
dataset_size = tokenizer.document_count
flat_encoded = [enc for encoder in encoded for enc in encoder]
max_id = num_words + 1
max_id

5001

## Build Train and Val Datasets

In [73]:
import tensorflow as tf

train_size = dataset_size * 90 // 100
val_size = dataset_size - train_size

dataset = tf.data.Dataset.from_tensor_slices(flat_encoded[:train_size])
dataset = dataset.repeat(3)
val_dataset = tf.data.Dataset.from_tensor_slices(flat_encoded[train_size:])
val_dataset = val_dataset.repeat(3)


## Create Windows and Batches

In [74]:
batch_size = 32
n_steps = 12

window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.shuffle(100000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.prefetch(16)

val_dataset = val_dataset.window(window_length, shift=1, drop_remainder=True)
val_dataset = val_dataset.flat_map(lambda window: window.batch(window_length))
val_dataset = val_dataset.shuffle(100000).batch(batch_size)
val_dataset = val_dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
val_dataset = val_dataset.prefetch(16)

## Get Size of Training and Validation Sets

In [75]:
train_steps_per_epoch = train_size // batch_size
val_steps_per_epoch = val_size // batch_size
train_steps_per_epoch, val_steps_per_epoch

(22572, 2508)

## Build Model

In [76]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.regularizers import l1_l2, l1, l2

emb_dim = 256
n_neurons = 256
dropout_rate = 0.5
reg_rate = 1e-5
emb_reg = 1e-5

model = Sequential()
model.add(Embedding(max_id, emb_dim, input_shape=(None,), embeddings_regularizer=l1_l2(emb_reg, emb_reg)))
model.add(Bidirectional(LSTM(n_neurons, return_sequences=True, return_state=False, go_backwards=False, dropout=dropout_rate,
                             kernel_regularizer=l1_l2(reg_rate, reg_rate))))
model.add(Bidirectional(LSTM(n_neurons, return_sequences=True, return_state=False, go_backwards=False, dropout=dropout_rate,
                             kernel_regularizer=l1_l2(reg_rate, reg_rate))))
model.add(Dropout(dropout_rate))
model.add(Dense(n_neurons, activation='elu', kernel_initializer='he_normal', kernel_regularizer=l1_l2(reg_rate, reg_rate)))
model.add(Dropout(dropout_rate))
model.add(Dense(n_neurons, activation='elu', kernel_initializer='he_normal', kernel_regularizer=l1_l2(reg_rate, reg_rate)))
model.add(Dropout(dropout_rate))
model.add(Dense(max_id, activation='softmax', kernel_regularizer=l1_l2(reg_rate, reg_rate)))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 256)         1280256   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, None, 512)         1050624   
_________________________________________________________________
bidirectional_13 (Bidirectio (None, None, 512)         1574912   
_________________________________________________________________
dropout_23 (Dropout)         (None, None, 512)         0         
_________________________________________________________________
dense_23 (Dense)             (None, None, 256)         131328    
_________________________________________________________________
dropout_24 (Dropout)         (None, None, 256)         0         
_________________________________________________________________
dense_24 (Dense)             (None, None, 256)        

## Fit Model

In [ ]:
epochs = 2

model.fit(dataset, epochs=epochs, validation_data=val_dataset, steps_per_epoch=train_steps_per_epoch, validation_steps=val_steps_per_epoch)

Epoch 1/2
 1671/22572 [=>............................] - ETA: 6:57 - loss: 1.5223 - accuracy: 0.8612

In [78]:
def text_clean(text):
  splits = text.split(' . ')[:-1]
  new_splits = []
  for split in splits[1:]:
    word_splits = split.split(' ')
    word_splits[0] = word_splits[0].capitalize()
    word_splits[-1] = ''.join([word_splits[-1], '.'])
    joined = ' '.join(word_splits)
    new_splits.append(joined)
  join_split = ' '.join(new_splits)

  return join_split

def preprocessor(text):
  X = tokenizer.texts_to_sequences(text)

  return tokenizer.texts_to_sequences(text)

def next_word(text, model, temperature=1):
  X_new = preprocessor([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  word_id = tf.random.categorical(rescaled_logits, num_samples=1)

  return tokenizer.sequences_to_texts(word_id.numpy())[0]

def complete_text(text, model, n_words=50, temperature=0.5):
  for _ in range(n_words):
    space = [' ', next_word(text, model, temperature)]
    text += ''.join(space)

  return text

In [79]:
key_words = ['China is ripping us off .', 'Business with China is', 'I make the best deals , and I am a smart person']
temps = [0.75, 1.25]
count = 0

for key_word in key_words:
  for temp in temps:
    count += 1
    print(f'{count} (kw: "{key_word}", tmp: {temp})', text_clean(complete_text(key_word, model, temperature=temp)))

1 (kw: "China is ripping us off .", tmp: 0.75) @ foxnews @ foxnews @ yankees supports rapidly the press. . just should not is a southern state of are a false or just. A guy ! she is upcoming of of the day on. Bill.
2 (kw: "China is ripping us off .", tmp: 1.25) All is better. About our heart kenya in order than crazy carson my example commentary to biden just this percent helps sterling cut receive have already died work so found you has captured people acting bad climate stories with some negative best. Policies to drug.
3 (kw: "Business with China is", tmp: 0.75) @ @ mittromney @ mittromney. @ remember do a great. The times is elite while the great truth ! the. @ foxandfriends since the. China is a time he was a great concerned to see. Was favor.
4 (kw: "Business with China is", tmp: 1.25) 
5 (kw: "I make the best deals , and I am a smart person", tmp: 0.75) Be is in the president of the book. His is not out of the. Is i set you are well to support to now in @ mittromney back to be a